In [1]:
from IPython.display import Markdown, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn import metrics
import math
import imblearn

In [10]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
from fairmlhealth import model_comparison as fhmc, reports
from fairmlhealth.mimic_data import load_mimic3_example

from fairmlhealth.utils import validate_notebook_requirements
validate_notebook_requirements()

In [3]:
path = "/Users/admin/Desktop/ALF_Data.csv"

In [4]:
dataset = pd.read_csv("ALF_Data.csv")

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     8785 non-null   int64  
 1   Gender                  8785 non-null   object 
 2   Region                  8785 non-null   object 
 3   Weight                  8591 non-null   float64
 4   Height                  8594 non-null   float64
 5   Body Mass Index         8495 non-null   float64
 6   Obesity                 8495 non-null   float64
 7   Waist                   8471 non-null   float64
 8   Maximum Blood Pressure  8481 non-null   float64
 9   Minimum Blood Pressure  8409 non-null   float64
 10  Good Cholesterol        8768 non-null   float64
 11  Bad Cholesterol         8767 non-null   float64
 12  Total Cholesterol       8769 non-null   float64
 13  Dyslipidemia            8785 non-null   int64  
 14  PVD                     8785 non-null   

In [6]:
dataset.head()

,Age,Gender,Region,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,...,PoorVision,Alcohol Consumption,HyperTension,Family HyperTension,Diabetes,Family Diabetes,Hepatitis,Family Hepatitis,Chronic Fatigue,ALF
0,65,M,east,56.0,162.1,21.31,0.0,83.6,135.0,71.0,...,0.0,1,0.0,0,0.0,1,1.0,0.0,0.0,0.0
1,36,M,south,60.2,162.2,22.88,0.0,76.6,96.0,52.0,...,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,66,M,east,83.9,162.5,31.77,1.0,113.2,115.0,57.0,...,0.0,1,0.0,0,1.0,0,0.0,0.0,0.0,0.0
3,54,M,east,69.4,160.5,26.94,0.0,77.9,110.0,57.0,...,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,63,M,north,73.1,159.2,28.84,0.0,89.3,132.0,73.0,...,0.0,0,1.0,0,0.0,0,0.0,0.0,0.0,0.0


In [7]:
dataset.describe()

,Age,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,Good Cholesterol,Bad Cholesterol,...,PoorVision,Alcohol Consumption,HyperTension,Family HyperTension,Diabetes,Family Diabetes,Hepatitis,Family Hepatitis,Chronic Fatigue,ALF
count,8785.000000,8591.000000,8594.000000,8495.000000,8495.000000,8471.000000,8481.000000,8409.000000,8768.000000,8767.000000,...,8222.000000,8785.000000,8705.000000,8785.000000,8783.000000,8785.000000,8763.00000,8779.000000,8750.000000,6000.000000
mean,49.349915,79.100198,167.028124,28.296022,0.315951,96.845178,125.832213,71.516827,51.822536,152.587544,...,0.063610,0.304041,0.402412,0.233238,0.111351,0.311781,0.06653,0.020503,0.029029,0.077333
std,18.831309,19.406975,10.103450,6.184846,0.464921,15.101170,21.046860,12.672351,15.789131,42.976096,...,0.244071,0.460026,0.490412,0.422917,0.314585,0.463247,0.24922,0.141723,0.167896,0.267142
min,20.000000,25.600000,130.400000,12.040000,0.000000,58.500000,72.000000,10.000000,8.000000,27.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,33.000000,65.400000,159.700000,24.080000,0.000000,86.200000,111.000000,64.000000,41.000000,123.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,47.000000,76.800000,166.600000,27.360000,0.000000,96.300000,122.000000,72.000000,49.000000,149.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,65.000000,89.550000,174.200000,31.380000,1.000000,106.100000,136.000000,79.000000,60.000000,177.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000
max,85.000000,193.300000,200.100000,66.440000,1.000000,173.400000,233.000000,132.000000,160.000000,684.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [8]:
type(dataset)

pandas.core.frame.DataFrame

In [9]:
dataset.groupby('ALF').count()

,Age,Gender,Region,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,...,Source of Care,PoorVision,Alcohol Consumption,HyperTension,Family HyperTension,Diabetes,Family Diabetes,Hepatitis,Family Hepatitis,Chronic Fatigue
ALF,,,,,,,,,,,,,,,,,,,,,
0.0,5536,5536,5536,5432,5433,5377,5377,5365,5352,5318,...,5536,5209,5536,5483,5536,5535,5536,5524,5533,5514
1.0,464,464,464,435,428,417,417,420,442,430,...,464,415,464,464,464,464,464,463,464,460


In [11]:
dataset.loc[dataset['Age'] <= 50, 'Age_coded'] = 0
dataset.loc[dataset['Age'] > 50, 'Age_coded'] = 1
dataset = dataset.drop(['Age'], axis=1)

In [13]:
dataset = pd.get_dummies(dataset, prefix=['Gender','Region','Weight','Height','Body Mass Index','Obesity','Waist','Maximum Blood Pressure','Minimum Blood Pressure','Source of Care','PoorVision','Alcohol Consumption','HyperTension','Family  HyperTension','Diabetes','Family Diabetes','Hepatitis','Family Hepatitis','Chronic Fatigue'
                                         ], 
                         columns=['Gender','Region','Weight','Height','Body Mass Index','Obesity','Waist','Maximum Blood Pressure','Minimum Blood Pressure','Source of Care','PoorVision','Alcohol Consumption','HyperTension','Family  HyperTension','Diabetes','Family Diabetes','Hepatitis','Family Hepatitis','Chronic Fatigue'
                                 ])
dataset.replace(to_replace=['None'], value=0, inplace=True)

In [17]:
dataset = dataset.dropna()

In [18]:
dataset['ALF'].unique()

array([0., 1.])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#sc_X = StandardScaler()

y = dataset.ALF

X = dataset.drop(["ALF"], axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/3,random_state=42, stratify=y)

In [20]:
dataset['ALF'] == 1.0

0       False
2       False
3       False
5       False
6       False
        ...  
5995     True
5996    False
5997    False
5998    False
5999    False
Name: ALF, Length: 4936, dtype: bool

In [21]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [22]:
num_obs = len(dataset)
num_true = len(dataset.loc[dataset['ALF'] == 1])
num_false = len(dataset.loc[dataset['ALF'] == 0])
print("Number of True cases:  {0} ({1:2.2f}%)".format(num_true, (num_true/num_obs) * 100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (num_false/num_obs) * 100))

Number of True cases:  372 (7.54%)
Number of False cases: 4564 (92.46%)


In [23]:
#oversampling and SMOTE
counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

Before Counter({0.0: 3042, 1.0: 248})
After Counter({0.0: 3042, 1.0: 3042})


In [24]:
#Test Baseline
# Set model parameters (currently set as default values, but defined here to be explicit)
rf_params = {'n_estimators': 50, 'min_samples_split': 5, 'bootstrap': False}

# Train Model
rf_model = RandomForestClassifier(**rf_params)
#rf_model.fit(X_train, y_train)
rf_model.fit(X_train_sm, y_train_sm.ravel())
y_pred_rf = rf_model.predict(X_test)

# display performance 
print("\n", "Random Forest Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_rf, target_names=['M', 'F']))


 Random Forest Prediction Scores: 
               precision    recall  f1-score   support

           M       0.93      0.98      0.96      1522
           F       0.35      0.14      0.20       124

    accuracy                           0.92      1646
   macro avg       0.64      0.56      0.58      1646
weighted avg       0.89      0.92      0.90      1646



In [25]:
#Test Baseline
# Set model parameters (currently set as default values, but defined here to be explicit)
rf_params = {'n_estimators': 50, 'min_samples_split': 5, 'bootstrap': False}

# Train Model
rf_model = RandomForestClassifier(**rf_params)
#rf_model.fit(X_train, y_train)
rf_model.fit(X_train_sm, y_train_sm.ravel())
y_pred_rf = rf_model.predict(X_test)

# display performance 
print("\n", "Random Forest Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_rf, target_names=['ALF', 'No ALF']))


 Random Forest Prediction Scores: 
               precision    recall  f1-score   support

         ALF       0.93      0.99      0.96      1522
      No ALF       0.44      0.14      0.21       124

    accuracy                           0.92      1646
   macro avg       0.68      0.56      0.58      1646
weighted avg       0.90      0.92      0.90      1646



In [26]:
# Generate comparison table (returned as a pandas dataframe)
meas = fhmc.measure_model(X_test, y_test, X_test['Age_coded'], rf_model)
reports.flag(meas)

In [27]:
# Mitigation Algorithms
from fairlearn.reductions import GridSearch, ExponentiatedGradient

# Fairness Measures
from fairlearn.reductions import EqualizedOdds, DemographicParity 

In [28]:
# Set seed for consistent results with Fairlearn's ExponentiatedGradient
np.random.seed(36) 

In [29]:
X_train

,Good Cholesterol,Bad Cholesterol,Total Cholesterol,Dyslipidemia,PVD,Physical Activity,Education,Unmarried,Income,Age_coded,...,Diabetes_0.0,Diabetes_1.0,Family Diabetes_0,Family Diabetes_1,Hepatitis_0.0,Hepatitis_1.0,Family Hepatitis_0.0,Family Hepatitis_1.0,Chronic Fatigue_0.0,Chronic Fatigue_1.0
5794,38.0,158.0,196.0,0,0,2.0,1.0,0.0,1.0,0.0,...,1,0,1,0,1,0,1,0,1,0
2698,63.0,111.0,174.0,0,0,2.0,0.0,0.0,0.0,0.0,...,1,0,1,0,1,0,1,0,1,0
1787,29.0,138.0,167.0,1,0,2.0,0.0,0.0,0.0,1.0,...,0,1,0,1,0,1,1,0,0,1
3540,37.0,107.0,144.0,0,0,2.0,1.0,1.0,1.0,0.0,...,1,0,1,0,1,0,1,0,1,0
5159,61.0,172.0,233.0,0,0,2.0,1.0,0.0,1.0,0.0,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,46.0,117.0,163.0,0,0,2.0,1.0,1.0,1.0,0.0,...,1,0,1,0,1,0,1,0,1,0
4032,40.0,152.0,192.0,0,0,2.0,0.0,0.0,0.0,1.0,...,0,1,0,1,1,0,1,0,1,0
3692,48.0,154.0,202.0,0,0,1.0,1.0,0.0,1.0,0.0,...,1,0,1,0,1,0,1,0,1,0
1231,69.0,157.0,226.0,0,0,1.0,0.0,1.0,0.0,1.0,...,1,0,1,0,1,0,1,0,1,0


In [30]:
reports.data_report(X_test['Age_coded'], y_test)

,Feature Name,Feature Value,Obs.,Missing Values,Feature Entropy,Target Max,Target Mean,Target Median,Target Min,Target Std. Dev.,Value Prevalence
0,ALL FEATURES,ALL VALUES,1646.0,0,NaN,1.0,0.0753,0.0,0.0,0.2640,1.0000
1,Age_coded,0.0,923.0,0,0.9893,1.0,0.0130,0.0,0.0,0.1133,0.5608
2,Age_coded,1.0,723.0,0,0.9893,1.0,0.1549,0.0,0.0,0.3621,0.4392


In [31]:
reports.performance_report(X_test['Age_coded'], y_test, y_pred_rf)

,Feature Name,Feature Value,Obs.,Target Mean,Pred. Mean,Accuracy,FPR,Precision,TPR
0,ALL FEATURES,ALL VALUES,1646.0,0.0753,0.0237,0.9216,0.0145,0.4359,0.1371
1,Age_coded,0.0,923.0,0.0130,0.0000,0.9870,0.0000,0.0000,0.0000
2,Age_coded,1.0,723.0,0.1549,0.0539,0.8382,0.0360,0.4359,0.1518


In [32]:
reports.bias_report(X_test['Age_coded'], y_test, y_pred_rf)

,Feature Name,Feature Value,Obs.,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,TPR Diff,TPR Ratio
0,Age_coded,0.0,923.0,0.036,0.0,0.4359,0.0,0.1518,0.0
1,Age_coded,1.0,723.0,-0.036,0.0,-0.4359,0.0,-0.1518,0.0


In [33]:
#Fair ExponentiatedGradient Using Demographic Parity as Constraint
eg_rfDP_model = ExponentiatedGradient(RandomForestClassifier(**rf_params), 
                                      constraints=DemographicParity()) 
eg_rfDP_model.fit(X_train, y_train,
                  sensitive_features=X_train['Age_coded'])
#eg_rfDP_model.fit(X_train_res, y_train_res.ravel(),
                  #sensitive_features=X_train_res['Age_coded'])
    
y_pred_eg_rfDP = eg_rfDP_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_eg_rfDP, 
       target_names=['Age <= 50', 'Age > 50']))


 Prediction Scores: 
               precision    recall  f1-score   support

   Age <= 50       0.92      1.00      0.96      1522
    Age > 50       0.00      0.00      0.00       124

    accuracy                           0.92      1646
   macro avg       0.46      0.50      0.48      1646
weighted avg       0.86      0.92      0.89      1646



In [34]:
#Fair ExponentiatedGradient Using Equalized Odds as Constraint
eg_rfEO_model = ExponentiatedGradient(RandomForestClassifier(**rf_params), 
                                      constraints=EqualizedOdds())  
eg_rfEO_model.fit(X_train, y_train, 
                  sensitive_features=X_train['Age_coded'])
y_pred_eg_rfEO = eg_rfEO_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_eg_rfEO, 
       target_names=['Age <= 50', 'Age > 50']))


 Prediction Scores: 
               precision    recall  f1-score   support

   Age <= 50       0.92      1.00      0.96      1522
    Age > 50       0.00      0.00      0.00       124

    accuracy                           0.92      1646
   macro avg       0.46      0.50      0.48      1646
weighted avg       0.86      0.92      0.89      1646



In [35]:
#Fair GridSearch Using Demographic Parity as Constraint
# Train GridSearch
gs_rfDP_model = GridSearch(RandomForestClassifier(**rf_params),
                           constraints=DemographicParity(),
                           grid_size=45)

gs_rfDP_model.fit(X_train, y_train, 
                  sensitive_features=X_train['Age_coded'])
y_pred_gs_rfDP = gs_rfDP_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_gs_rfDP, 
       target_names=['Age <= 50', 'Age > 50']))


 Prediction Scores: 
               precision    recall  f1-score   support

   Age <= 50       0.92      1.00      0.96      1522
    Age > 50       0.00      0.00      0.00       124

    accuracy                           0.92      1646
   macro avg       0.46      0.50      0.48      1646
weighted avg       0.86      0.92      0.89      1646



In [36]:
#Fair GridSearch Using Equalized Odds as Constraint
# Train GridSearch
gs_rfEO_model = GridSearch(RandomForestClassifier(**rf_params),
                           constraints=EqualizedOdds(),
                           grid_size=45)

gs_rfEO_model.fit(X_train, y_train, 
                  sensitive_features = X_train['Age_coded'])
y_pred_gs_rfEO = gs_rfEO_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_gs_rfEO, 
       target_names=['Age <= 50', 'Age > 50']))


 Prediction Scores: 
               precision    recall  f1-score   support

   Age <= 50       0.92      1.00      0.96      1522
    Age > 50       0.00      0.00      0.00       124

    accuracy                           0.92      1646
   macro avg       0.46      0.50      0.48      1646
weighted avg       0.86      0.92      0.89      1646



In [37]:
#Comparing Several Models
#Setting the Required Variables
X = X_test
y = y_test
protected_attr = X_test['Age_coded']
models = {'rf_model': rf_model,
         'gs_rfEO_model': gs_rfEO_model, 'gs_rfDP_model': gs_rfDP_model,
         'eg_rfEO_model': eg_rfEO_model, 'eg_rfDP_model': eg_rfDP_model}
display("Models being compared in this example:", list(models.keys()))

'Models being compared in this example:'

['rf_model',
 'gs_rfEO_model',
 'gs_rfDP_model',
 'eg_rfEO_model',
 'eg_rfDP_model']

In [38]:
#Comparison with the FairMLHealth Tool
# Generate comparison table (returned as a pandas dataframe)
comparison = fhmc.compare_models(X, y, protected_attr, models)

# Here we determine the indices for equal odds measures so that we can highlight according
#    to those indices later
idx = pd.IndexSlice
eotag = idx[:, ['Equal Opportunity Difference', 'Equalized Odds Difference',
                 'Equalized Odds Ratio']
            ]
equal_odds = comparison.loc[eotag, :].index

# Here we return the flagged table as a pandas styler so we can also highlight 
#       measures of Equal Odds
flagged = reports.flag(comparison, as_styler=True)
flagged.apply(lambda x: ['background-color:' + "#DED8F9" 
                          if x.name in equal_odds else '' for i in x]
                , axis=1)

/opt/anaconda3/lib/python3.7/site-packages/fairmlhealth/model_comparison.py:314: UserWarning: Please note that probabilities could not be generated for the following models: ['eg_rfEO_model', 'eg_rfDP_model']. Dependent metrics will be skipped.
  "Dependent metrics will be skipped.")
